# PCP case study

This case study concerns the transient difference absorption measured after 490 nm
excitation from the peridinin chlorophyll protein (abbreviated PCP), a light-harvesting
complex of oceanic photosynthetic algae. The measurement and the global analysis are described in (van Stokkum et al. 2009), Fig.S6. The target analysis is based upon the model explained in (van Stokkum et al. 2009), Fig.4 and Fig.7.

van Stokkum IHM, Papagiannakis E, Vengris M, Salverda JM, Polivka T, Zigmantas D, Larsen DS, Lampoura SS, Hiller RG, van Grondelle R (2009) Inter-pigment interactions in the peridinin chlorophyll protein studied by global and target analysis of time resolved absorption spectra. Chemical Physics 357 (1-3):70-78.

### Inspect data

In [ ]:
from pyglotaran_extras import plot_data_overview

PCP_data_path = "PCP490sel.ascii"
plot_data_overview(PCP_data_path, nr_of_data_svd_vectors=5);

## Global Analysis

### Used model and parameters

In [ ]:
from glotaran.utils.ipython import display_file

PCP_global_model_path = "models/PCP-global-model.yml"
PCP_global_parameters_path = "models/PCP-global-parameters.yml"

#### Model file

In [ ]:
display_file(PCP_global_model_path, syntax="yml")

#### Parameters file

In [ ]:
display_file(PCP_global_parameters_path, syntax="yml")

The seven free parameters are the center (the location of the maximum of the IRF) and the width of the IRF, and the five kinetic parameters.
Note that these data have been precorrected for dispersion. Dispersion parameters could have been estimated from the uncorrected data as well.

### Create scheme and optimize it

In [ ]:
from glotaran.analysis.optimize import optimize
from glotaran.project.scheme import Scheme

PCP_global_scheme = Scheme(
    model=PCP_global_model_path,
    parameters=PCP_global_parameters_path,
    data={"PCP_data": PCP_data_path},
)
PCP_global_scheme.validate()

In [ ]:
PCP_global_result = optimize(PCP_global_scheme)

In [ ]:
PCP_global_result

In [ ]:
PCP_global_result.optimized_parameters

### Result plots

#### Fit quality

In [ ]:
from pyglotaran_extras import plot_fitted_traces, select_plot_wavelengths

wavelengths = select_plot_wavelengths(PCP_global_result.data["PCP_data"])
plot_fitted_traces(PCP_global_result.data["PCP_data"], wavelengths);

In [ ]:
fig_global, axes = plot_fitted_traces(
    PCP_global_result.data["PCP_data"], wavelengths, linlog=True, linthresh=1
)

Note that in the second plot the time axis is linear until 1 ps and logarithmic thereafter. This plot is more informative.

#### Overview

In [ ]:
from pyglotaran_extras import plot_overview

plot_overview(
    PCP_global_result.data["PCP_data"],
    linlog=True,
    figure_only=False,
    nr_of_data_svd_vectors=5,
    nr_of_residual_svd_vectors=1,
);

## Target Analysis

### Used model and parameters

In [ ]:
PCP_target_model_path = "models/PCP-target-model.yml"
PCP_target_parameters_path = "models/PCP-target-parameters.yml"

#### Model file

In [ ]:
display_file(PCP_target_model_path, syntax="yml")

#### Parameters file

In [ ]:
display_file(PCP_target_parameters_path, syntax="yml")

The free parameters are the center (the location of the maximum of the IRF) and the width of the IRF, and some of the kinetic parameters.
Which kinetic parameters can be estimated is difficult, and requires some trial and error. Six eigenvalues (the reciprocals of the lifetimes) of the K matrix can in principle be estimated and spectral assumptions are needed for the remaining kinetic parameters. Because of the limited time range measured(less than 100 ps), it is better to fix the Chl decay rate at 1/(2ns).

### Create scheme and optimize it

In [ ]:
PCP_target_scheme = Scheme(
    model=PCP_target_model_path,
    parameters=PCP_target_parameters_path,
    maximum_number_function_evaluations=1,
    data={"PCP_data": PCP_data_path},
)
PCP_target_scheme.validate()

In [ ]:
PCP_target_result = optimize(PCP_target_scheme)

In [ ]:
PCP_target_result

### Result plots

#### Fit quality

In [ ]:
fig_target, axes = plot_fitted_traces(
    PCP_target_result.data["PCP_data"], wavelengths, linlog=True, linthresh=1
);

#### Overview

In [ ]:
plot_overview(
    PCP_target_result.data["PCP_data"],
    figure_only=False,
    nr_of_data_svd_vectors=3,
    nr_of_residual_svd_vectors=1,
    linlog=True,
    linthresh=1,
);

In [ ]:
PCP_target_result.optimized_parameters

In [ ]:
compartments = PCP_target_scheme.model.initial_concentration["input1"].compartments

PCP_target_scheme.model.k_matrix["km1"].matrix_as_markdown(compartments).replace("0.0000e+00", "")

In [ ]:
PCP_target_scheme = Scheme(
    model=PCP_target_model_path,
    parameters=PCP_target_parameters_path,
    maximum_number_function_evaluations=4,
    data={"PCP_data": PCP_data_path},
)
PCP_target_scheme.validate()

In [ ]:
PCP_target_result2 = optimize(PCP_target_scheme)

In [ ]:
PCP_target_result2

In [ ]:
plot_overview(
    PCP_target_result2.data["PCP_data"],
    figure_only=False,
    nr_of_data_svd_vectors=3,
    nr_of_residual_svd_vectors=1,
    linlog=True,
    linthresh=1,
);

After optimization the residuals have improved. However, the interpretability of the SADS of slowS1 (green) and to a lesser extent also S1 (blue) have deteriorated. Some more fine tuning with the help of spectral assumptions is needed. The interplay of the decay of slowS1 (via EET to Chl, and to GS), and Chl* annihilation is difficult to resolve.